# **Install Libraries**


In [ ]:

!pip install -q transformers
!pip install pypdf
!pip install tiktoken
!pip install langchain
!pip install -U langchain-community
!pip install -U sentence-transformers
!pip install faiss-cpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 15.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_

# **Load model**


In [ ]:
import torch
from transformers import AutoTokenizer

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive/')

# Define the folder path and model path
folder_path = '/content/drive/My Drive/Colab Notebooks/COMP8420/Major Project'
model_path = os.path.join(folder_path, 'model.model')

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(folder_path, use_fast=True)

# Check if CUDA (GPU) is available and set the device accordingly
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Load the model to the appropriate device
model = torch.load(model_path, map_location=device)

# Verify the loading
print("Tokenizer and model loaded successfully.")

# Print the model's structure
print(model)


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Using device: cpu
Tokenizer and model loaded successfully.
DistilBertForQuestionAnswering(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=

# **Load and process PDF file**

In [ ]:
# Import necessary libraries
import langchain
print("langchain installed successfully")

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define the paths to your PDF files in Google Drive
pdf_paths = [
    '/content/drive/My Drive/Colab Notebooks/COMP8420/Major Project/Paper 23.pdf',
]

# Use PyPDFLoader to load and process the PDF files
from langchain.document_loaders import PyPDFLoader

# Initialize the loaders
loaders = [PyPDFLoader(pdf_path) for pdf_path in pdf_paths]

# Load the pages from the PDF files
pages = []
for loader in loaders:
    pages.extend(loader.load())




langchain installed successfully
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Split the loaded text into smaller chunks for processing
from langchain.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(chunk_size=128, chunk_overlap=12)
docs = text_splitter.split_documents(pages)

# Import necessary libraries for embeddings
import numpy as np
from langchain_community.embeddings import HuggingFaceEmbeddings

# Initialize the HuggingFaceEmbeddings model
encoder = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L12-v2', model_kwargs={'device': "cpu"})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Embed queries for comparison
embeddings1 = encoder.embed_query("RAG")
embeddings2 = encoder.embed_query(docs[0].page_content)

# Calculate and print the dot product of the embeddings
print(np.dot(embeddings1, embeddings2))

# Import FAISS for vector storage
from langchain.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy

# Initialize FAISS database with the documents and embeddings
faiss_db = FAISS.from_documents(docs, encoder, distance_strategy=DistanceStrategy.DOT_PRODUCT)

-0.1044129763190903


# **Load model**


In [ ]:
# Additional imports for question answering
import torch
from transformers import AutoTokenizer, DistilBertForQuestionAnswering

# Load the tokenizer and model from Google Drive
folder_path = '/content/drive/My Drive/Colab Notebooks/COMP8420/Major Project'
model_path = os.path.join(folder_path, 'model.model')

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(folder_path, use_fast=True)

# Check if CUDA (GPU) is available and set the device accordingly
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Load the model to the appropriate device
model = torch.load(model_path, map_location=device)
model.to(device)

# Verify the loading
print("Tokenizer and model loaded successfully.")

Using device: cpu
Tokenizer and model loaded successfully.


In [ ]:
# Define a function to perform question answering
def answer_question(question, context):
    inputs = tokenizer(question, context, return_tensors='pt')
    inputs = {key: value.to(device) for key, value in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))
    return answer


# **Generate answers**


Try a simple question "What is the purpose of this paper?"

In [ ]:
# Example usage with retrieved documents from FAISS
question = "What is the purpose of this paper?"
retrieved_docs = faiss_db.similarity_search(question, k=10)
context = "".join(doc.page_content + "\n" for doc in retrieved_docs)
torch.cuda.empty_cache()

# Truncate the context to fit within the model's token limit
context = context[:tokenizer.model_max_length]

# Answer the question based on the retrieved context
answer = answer_question(question=question, context=context)
print(f"Question: {question}")
print(f"Answer: {answer}")

Question: What is the purpose of this paper?
Answer: rightsholder ( s ) ; author self - archiving of the accepted manuscript version of this article is solely governed by the terms of such publishing agreement and applicable law. s acclaimed gti appears associated with diminished dedication to csr such as transparent disclosure and protecting consumer rights.


The answer is not make sence, so I print the context that the model used to generate answer to see any problem from generating context.

In [ ]:
# Example usage with retrieved documents from FAISS
question = "What is the purpose of this paper?"
retrieved_docs = faiss_db.similarity_search(question, k=10)
context = "".join(doc.page_content + "\n" for doc in retrieved_docs)
torch.cuda.empty_cache()

# Print the context
print(f"Context: {context}")

# Ensure the context fits within the model's token limit by splitting if necessary
inputs = tokenizer(question, context, return_tensors='pt', truncation=True, max_length=512)
input_ids = inputs['input_ids'][0]

# Create a list of contexts to process each part within the limit
context_splits = [tokenizer.decode(input_ids[i:i + 512]) for i in range(0, len(input_ids), 512)]

# Iterate through context splits to find the most relevant answer
answers = []
for split in context_splits:
    answer = answer_question(question=question, context=split)
    answers.append(answer)

# Join the answers from all splits (if required) and print the final answer
final_answer = " ".join(answers)
print(f"Question: {question}")
print(f"Answer: {final_answer}")


Context:  rightsholder(s); author self-archiving of the accepted 
manuscript version of this article is solely governed by the terms of 
such publishing agreement and applicable law.
�s acclaimed GTI 
appears associated with diminished dedication to CSR such 
as transparent disclosure and protecting consumer rights.For any questions related to the empirical part please email the 
corresponding author Congshan Li at congshan.li@xmu.edu.cn and 
for questions related to the theory part please email Yuntao Bai at 
ytbai@xmu.edu.cn. 
 * Congshan Li 
 Congshan.li@xmu.edu.cn
* Yuntao Bai 
 ytbai@xmu
ability, 13(5), 2891.
Liu, D. S., Chen, J. K., & Zhang, N. (2021a). Political connections and 
green technology innovations under an environmental regulation. 
Journal of Cleaner Production, 298(May), 126778.
Loi, T. I., Kuhn, K. M., Sahaym, A., Butterfield, K. D., & Tripp, T. 
M. (2020). From helping hands to harmful acts: When and how 
employee volunteering promotes workplace deviance. Journal 


Try another question "What is Green technology innovation?"

In [ ]:
# Example usage with retrieved documents from FAISS
question = "What is Green technology innovation?"
retrieved_docs = faiss_db.similarity_search(question, k=10)
context = "".join(doc.page_content + "\n" for doc in retrieved_docs)
torch.cuda.empty_cache()

# Truncate the context to fit within the model's token limit
inputs = tokenizer(question, context, return_tensors='pt', truncation=True, max_length=512)

# Convert the input_ids back to text to make sure the context is within the token limit
context = tokenizer.decode(inputs['input_ids'][0], skip_special_tokens=True)

# Print the context
# print(f"Context: {context}")

# Answer the question based on the truncated context
answer = answer_question(question=question, context=context)
print(f"Question: {question}")
print(f"Answer: {answer}")


Question: What is Green technology innovation?
Answer: environmental regulation and innova - tion : a panel data study. review of economics and statistics, 79 ( 4 ), 610 – 619. parsons, c. a., sulaeman, j., & titman, s. ( 2018 ). the geogra - phy of financial misconduct.


The generated context does not always make sense, leading to nonsensical answers.

To test the practical application of the model in a real-world scenario, I will provide a specific context and generate an answer based on that.


In [ ]:
# Define the context directly
context = """
Green technology innovation (GTI), e.g., clean production
technologies to reduce greenhouse gas emissions, refers to
activities that upgrade technologies or processes to conserve
resources or safeguard the environment (Brunnermeier &
Cohen, 2003; Chen et al., 2006). GTI is considered part
of the global movement toward environmental sustainability because it integrates environmental objectives into
the market activities of firms, thereby providing economic
benefits while enhancing their environmental legitimacy
(El-Kassar & Singh, 2019; Zhang et al., 2019). Most
research and reports have focused on the positive effects
of GTI on firms and industries, such as corporate financial
performance and industry evolution (e.g., Du et al., 2021;
Tang et al., 2012). However, anecdotes demonstrate a caveat
to this generalization: GTI may also have a dark side.
"""

# Define the question
question = "What is green technology innovation?"

# Ensure the context fits within the model's token limit
inputs = tokenizer(question, context, return_tensors='pt', truncation=True, max_length=512)

# Convert the input_ids back to text to make sure the context is within the token limit
context = tokenizer.decode(inputs['input_ids'][0], skip_special_tokens=True)

# Print the context
# print(f"Context: {context}")

# Answer the question based on the truncated context
answer = answer_question(question=question, context=context)
print(f"Question: {question}")
print(f"Answer: {answer}")


Question: What is green technology innovation?
Answer: green technology innovation


In [ ]:
# Define the new context directly
context = """
Green technology innovation (GTI) has been increasingly adopted by firms worldwide to promote sustainable development,
whereas its potential downsides have been largely overlooked. Drawing on moral licensing theory, we devise a framework
to reveal the potential dark side of firms’ GTI on corporate social responsibility (CSR). We argue that with the global eco-
awakening, GTI has been an efficient means for firms to meet their stakeholders’ expectations and environmental legitimacy.
This may cause a moral licensing effect for senior executives’ ethical strategic decision-making, due to their belief that
existing GTI could balance the firm’s subsequent moral neglect, thereby minimizing firms’ CSR. Using the Chinese listed
manufacturing firms as a sample, we find empirical support for our prediction. In addition, we investigate how firms’ envi-
ronmental awards and honors may exacerbate the negative impact of GTI on CSR, whereas the presence of women executives
may mitigate the relation. Our findings can trigger firms’ reflections on GTI’s downsides, and contribute to research and
practice in comprehensively understanding firms’ GTI consequences, CSR decisions, and moral balance.
"""

# Define the question
question = "What is the finding of this paper?"

# Ensure the context fits within the model's token limit
inputs = tokenizer(question, context, return_tensors='pt', truncation=True, max_length=512)

# Convert the input_ids back to text to make sure the context is within the token limit
context = tokenizer.decode(inputs['input_ids'][0], skip_special_tokens=True)

# Print the context
print(f"Context: {context}")

# Answer the question based on the truncated context
answer = answer_question(question=question, context=context)
print(f"Question: {question}")
print(f"Answer: {answer}")


Context: what is the finding of this paper? green technology innovation ( gti ) has been increasingly adopted by firms worldwide to promote sustainable development, whereas its potential downsides have been largely overlooked. drawing on moral licensing theory, we devise a framework to reveal the potential dark side of firms ’ gti on corporate social responsibility ( csr ). we argue that with the global eco - awakening, gti has been an efficient means for firms to meet their stakeholders ’ expectations and environmental legitimacy. this may cause a moral licensing effect for senior executives ’ ethical strategic decision - making, due to their belief that existing gti could balance the firm ’ s subsequent moral neglect, thereby minimizing firms ’ csr. using the chinese listed manufacturing firms as a sample, we find empirical support for our prediction. in addition, we investigate how firms ’ envi - ronmental awards and honors may exacerbate the negative impact of gti on csr, whereas t

The answer is still not making sense. Therefore, I conclude that this model can generate short answers from the context but not comprehensive answers. As a result, this model does not work well for answering questions from uploaded articles.
